In [4]:
import sys
sys.path.append('../')

import numpy as np
from pyqubo import Array
from qubobrute import *


from numba import cuda

cuda.is_available()



In [2]:
def qubo_energy(qubo: np.ndarray, offset: np.number, sample: np.ndarray) -> np.number:
    """Calculate the energy of a sample."""
    return np.dot(sample, np.dot(qubo, sample)) + offset

## Setup Small Problem

In [11]:
nbits = 25
x = Array.create("x", shape=(nbits,), vartype="BINARY")
H = (np.arange(nbits) @ x - 12) ** 2
model = H.compile()
qubo, offset = model.to_qubo(index_label=True)
q = to_mat(qubo)

## Bruteforce Solution on GPU

In [4]:
energies = solve_gpu(q, offset)
best = bits(energies.argmin(), nbits=nbits)
qubo_energy(q, offset, sample=best), best

(0.0,
 array([0., 1., 1., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32))

## Run Simulated Annealing on the CPU

In [9]:
energies, solutions = simulate_annealing(q, offset, n_iter=1000, n_samples=100_000, temperature=1.0, cooling_rate=0.99)

best = solutions[energies.argmin()]
energies.min(), best

(0.0,
 array([0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 1], dtype=int8))

## Run Simulated Annealing on the GPU

In [10]:
energies, solutions = simulate_annealing_gpu(q, offset, n_iter=1000, n_samples=100_000, temperature=1.0, cooling_rate=0.99)

best = solutions[energies.argmin()]
energies.min(), best

(0.0,
 array([1., 1., 0., 0., 0., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32))